# Advanced Querying Mongo

Importing libraries and setting up connection

In [24]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
cursor = MongoClient('mongodb://localhost:27017')
client = cursor.companies
comps = client.companies

In [34]:
list(comps.find().limit(1))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [37]:
query = {'name' : 'Babelgum'}
select = {'_id': 0 , 'name':1}

list(comps.find(query,select))


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [42]:
query = {'number_of_employees' : {'$gt' : 5000}}
select = {'_id': 0 , 'name':1, 'number_of_employees':1}

list(comps.find(query,select).sort('number_of_employees',-1).limit(5))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [50]:
query = {'$and':[{'founded_year' : {'$gte' : 2000}},{'founded_year' : {'$lte' : 2005}}]}
select = {'_id': 0 , 'name':1, 'founded_year':1}

list(comps.find(query,select).limit(5))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [61]:
query = {'$and':[{'ipo.valuation_amount' : {'$gte' : 100000000}},{'founded_year' : {'$lt' : 2010}}]}
select = {'_id': 0 , 'name':1,'ipo' : 1}

list(comps.find(query,select).limit(3))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [63]:
query = {'$and':[{'number_of_employees' : {'$lt' : 1000}},{'founded_year' : {'$lt' : 2005}}]}
select = {'_id': 0 , 'name':1}

list(comps.find(query,select).sort('number_of_employees',-1).limit(5))

[{'name': 'Infinera Corporation'},
 {'name': 'NorthPoint Communications Group'},
 {'name': '888 Holdings'},
 {'name': 'Forrester Research'},
 {'name': 'Webmetrics'}]

### 6. All the companies that don't include the `partners` field.

In [70]:
query = {'partners':{'$exists' : False}}
select = {'_id': 0 , 'name':1}

list(comps.find(query,select).sort('number_of_employees',-1).limit(5))
#empty

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [80]:
query = {'category_code':{'$type' : 10}}
select = {'_id': 0 , 'name':1,'category_code':1}

list(comps.find(query,select).limit(5))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [81]:
query = {'$and':[{'number_of_employees' : {'$gte' : 100}},{'number_of_employees': {'$lt' : 1000}}]}
select = {'_id': 0 , 'name':1,'number_of_employees':1}

list(comps.find(query,select).limit(5))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [83]:
query = {}
select = {'_id': 0 , 'name':1,'ipo.valuation_amount' : 1}

list(comps.find(query,select).sort('ipo.valuation_amount', -1).limit(5))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [84]:
query = {}
select = {'_id': 0 , 'name':1,'number_of_employees':1}

list(comps.find(query,select).sort('number_of_employees',-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [89]:
query = {'$and':[{'founded_month' : {'$gte': 7}},{'founded_month' : {'$lte': 12}}]}
select = {'_id': 0 , 'name':1,'founded_month':1}

list(comps.find(query,select))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'delicious', 'founded_month': 9},
 {'name': 'AllPeers', 'founded_month': 8},
 {'name': 'Wize', 'founded_month': 8},
 {'name': 'SellABand', 'founded_month': 8},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'MeeVee', 'foun

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [99]:
query = {'$and':[{'founded_year' : {'$lt': 2000}},{'acquisition.price_amount' : {'$gt': 10000000}}]}
select = {'_id': 0 , 'name':1,'acquisition.price_amount' : 1}

list(comps.find(query,select).limit(5))

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish', 'acquisition': {'price_amount': 300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [104]:
query = {'acquisition.acquired_year' : {'$gt': 2010}}
select = {'_id': 0 , 'name':1,'acquisition' : 1}

list(comps.find(query,select).sort('acquisition.price_amount',-1).limit(2))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [106]:
query = {}
select = {'_id': 0 , 'name':1,'founded_year' : 1}

list(comps.find(query,select).sort('founded_year',-1).limit(5))

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [109]:
query = {'founded_day' : {'$lte': 7}}
select = {'_id': 0 , 'name':1}

list(comps.find(query,select).sort('acquisition.price_amount',-1).limit(5))

[{'name': 'Netscape'},
 {'name': 'PayPal'},
 {'name': 'Zappos'},
 {'name': 'Alibaba'},
 {'name': 'Postini'}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [112]:
query = {'$and':[{'category_code' : 'web'},{'number_of_employees' : {'$gt': 4000}}]}
select = {'_id': 0 , 'name':1,'number_of_employees' : 1}

list(comps.find(query,select).sort('number_of_employees', 1).limit(5))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Rakuten', 'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [118]:
query = {'$and':[{'acquisition.price_amount' : {'$gt': 77}},{'acquisition.price_currency_code': 'EUR'}]}
select = {'_id': 0 , 'name':1}

list(comps.find(query,select).limit(5))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Mobile Trend'},
 {'name': 'Citizenside'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [121]:
query = {'$and':[{'founded_month' : {'$gte': 1}},{'founded_month' : {'$lte': 3}}]}
select = {'_id': 0 , 'name':1,'acquisition':1}

list(comps.find(query,select).limit(2))

[{'name': 'Facebook', 'acquisition': None},
 {'name': 'Twitter', 'acquisition': None}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [126]:
query = {'$and':[{'founded_year' : {'$gte': 2000}},{'founded_year' : {'$lte': 2010}},{'acquisition.acquired_year':{'$not':{'$lt':2011} }} ] }
select = {'_id': 0 , 'name':1,'founded_month':1}

list(comps.find(query,select).limit(5))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Facebook', 'founded_month': 2},
 {'name': 'Omnidrive', 'founded_month': 11}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [170]:
query = {'deadpooled_year' : {'$gte' : 100}}
select = {'_id': 0 , 'name':1,'founded_year':1, 'deadpooled_year':1}
result = []
for i in comps.find(query,select):
    try:
        x = i['founded_year'] - i['deadpooled_year']
        if x == 3:
            result.append(i['name'])
    except:
        continue
result

['Fliggo']